# Kerchunk Reading Recipes
Here we give examples of different formats for Kerchunk which require specific Recipes to open.

Access Control:
 - No credentials
 - Certificate Credentials
 - Token Credentials

Link Type:
 - Local Kerchunk File
 - Kerchunk over HTTPS (Kerchunk file itself is HTTPS link)
 - Kerchunk via S3

Kerchunk Type:
 - Standard Kerchunk JSON
 - Kerchunk ZSTD
 - Kerchunk Parquet

See https://stfc365-my.sharepoint.com/:x:/r/personal/daniel_westwood_stfc_ac_uk/_layouts/15/Doc.aspx?sourcedoc=%7B1A4A6C96-4291-40D7-A6E3-4665AEA57EF1%7D&file=Book.xlsx&action=editnew&mobileredirect=true&wdNewAndOpenCt=1696858849368&ct=1696858849741&wdPreviousSession=d8a3c4f4-b716-409a-8793-00cba44b2d8f&wdOrigin=OFFICECOM-WEB.START.NEW&login_hint=daniel.westwood%40stfc.ac.uk&cid=d063908b-7e3c-42c1-9bfb-f7c700f38fe7&wdPreviousSessionSrc=HarmonyWeb for a summary of known recipes

In [ ]:
import fsspec
import xarray as xr
import s3fs

import aiohttp
import asyncio
import requests
import ssl
from fsspec.implementations.reference import LazyReferenceMapper

## Access Control: No Credentials
Opening sequences for Kerchunk with No Access control

### Local K File Opening

In [ ]:
# Standard Kerchunk JSON
kfile  = 'kerchunk.json'
mapper = fsspec.get_mapper(
    'reference://', # Required
    fo=kfile,       # Required
    **get_mapper_kwargs)

ds = xr.open_zarr(
    kfile,          # Required
    **open_zarr_kwargs) 

# Requires get_mapper_kwargs, open_zarr_kwargs (consolidated=False, decode_times=False, decode_timedelta=False)

In [ ]:
# Kerchunk ZSTD
kzstd  = 'kerchunk.zst'
mapper = fsspec.get_mapper(
    'reference://', # Required
    fo=kzstd,       # Required
    target_options={'compression':'zstd'}  # Required
    **get_mapper_kwargs)

ds = xr.open_zarr(
    kfile,          # Required
    **open_zarr_kwargs) 

# Requires get_mapper_kwargs, open_zarr_kwargs (consolidated=False, decode_times=False, decode_timedelta=False)

In [ ]:
# Kerchunk Parquet
kstore = 'kerchunk.parq'
fs = fsspec.implementations.reference.ReferenceFileSystem(
    kstore,                   # Required
    remote_protocol='file',   # Required
    target_protocol="file",   # Required
    lazy=True,                # Required
    **rfs_kwargs)

ds = xr.open_dataset(
    fs.get_mapper(),          # Required
    engine="zarr",            # Required
    **open_dataset_kwargs)

# Requires rfs_kwargs, open_dataset_kwargs (backend_kwargs={"consolidated": False, "decode_times": False})

## Kerchunk over HTTPS

## Kerchunk via S3

In [ ]:
# Standard Kerchunk JSON
kfile = "s3://test_bucket/kerchunk.json"
ref = s3fs.S3FileSystem(**fssopts)    # fssopts - key, secret, client_kwargs
ref = ref.open(kfile)                 # s3open  - compression=None

mapper = fsspec.get_mapper(
    "reference://", 
    fo=ref,
    target_protocol="http", 
    remote_options=fssopts, 
    target_options={"compression": None}
)
xobj = xr.open_zarr(mapper, **_xr_open_args)

# Requires s3fssopts_kwargs, get_mapper_kwargs, open_zarr_kwargs

In [ ]:
# Kerchunk ZSTD
kfile = "s3://test_bucket/kerchunk.zst"
ref = s3fs.S3FileSystem(**fssopts)    # fssopts - key, secret, client_kwargs
ref = ref.open(kfile)                 # s3open  - compression=None

mapper = fsspec.get_mapper(
    "reference://", 
    fo=ref,
    target_protocol="http", 
    remote_options=fssopts, 
    target_options={"compression": 'zstd'} # Simple change
)
xobj = xr.open_zarr(mapper, **_xr_open_args)

# Requires s3fssopts_kwargs, get_mapper_kwargs, open_zarr_kwargs

In [ ]:
# Kerchunk Parquet?
